In [50]:
import sys
from collections import defaultdict

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sqlite3 as sql

In [51]:
conn = sql.connect("dataset/database.sqlite")

In [52]:
df = pd.read_sql_query("select * from business", conn)
df.head()

,id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open
0,--6MefnULPED_I942VcFNA,John's Chinese BBQ Restaurant,,"328 Highway 7 E, Chalmers Gate 11, Unit 10",Richmond Hill,ON,L4B 3P7,43.8409,-79.3996,3.0,30,1
1,--7zmmkVg-IMGaXbuVd0SQ,Primal Brewery,,16432 Old Statesville Rd,Huntersville,NC,28078,35.4371,-80.8437,4.0,42,1
2,--8LPVSo5i0Oo61X01sV9A,Valley Bone and Joint Specialists,,"3941 E Baseline Rd, Ste 102",Gilbert,AZ,85234,33.3795,-111.7280,4.5,3,1
3,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,The Strip,3355 Las Vegas Blvd S,Las Vegas,NV,89109,36.1232,-115.1690,4.0,1389,1
4,--9QQLMTbFzLJ_oT-ON3Xw,Great Clips,,"1835 E Guadalupe Rd, Ste 106",Tempe,AZ,85283,33.3617,-111.9100,3.0,9,1


In [53]:
df_checkins = pd.read_sql_query("select * from checkin_total", conn)

In [54]:
df_checkins.head()

,business_id,total_count
0,7KPBkxAOEtb3QeIL9PEErg,113
1,kREVIrSBbtqBhIYkTccQUg,7
2,tJRDll5yqpZwehenzE2cSg,4
3,nhZ1HGWD8lMErdn3FuWuTQ,45
4,vDoXZGE7p6xAkKQ0XQPvoA,2


In [55]:
df_checkins[df_checkins.total_count > 100000]

,business_id,total_count
13280,FaHADZARwnY4yvlvpnsfGA,126689
23286,JmI9nslLD7KZqRr__Bg6NQ,107891


In [56]:
df[df.id == 'FaHADZARwnY4yvlvpnsfGA']

,id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open
52404,FaHADZARwnY4yvlvpnsfGA,McCarran International Airport,Southeast,5757 Wayne Newton Blvd,Las Vegas,NV,89119,36.085,-115.151,3.5,3090,1


In [57]:
df[df.id == 'JmI9nslLD7KZqRr__Bg6NQ']

,id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open
73791,JmI9nslLD7KZqRr__Bg6NQ,Phoenix Sky Harbor International Airport,,"3400 E Sky Harbor Blvd, Ste 3300",Phoenix,AZ,85034,33.4359,-112.011,3.0,2103,1


In [58]:
df_cat = pd.read_sql_query("select * from category", conn)


In [59]:
df_cat.head()

,business_id,category
0,YDf95gJZaq05wvo7hTQbbQ,Shopping
1,YDf95gJZaq05wvo7hTQbbQ,Shopping Centers
2,mLwM-h2YhXl2NCgdS84_Bw,Food
3,mLwM-h2YhXl2NCgdS84_Bw,Soul Food
4,mLwM-h2YhXl2NCgdS84_Bw,Convenience Stores


In [60]:
possible_cats = set(df_cat.category.unique())

In [61]:
airport_cats = []
for cat in possible_cats:
    if 'airport' in cat.lower():
        print(cat)
        airport_cats.append(cat)

Airport Lounges
Airport Terminals
Airports
Airport Shuttles


In [62]:
airports = []
for airport_cat in airport_cats:
    airports.extend(list(df_cat[df_cat.category == airport_cat].business_id))

In [63]:
print(len(airports))
print(len(set(airports)))

437
394


In [64]:
category_counts = defaultdict(int)

for cat in list(df_cat.category):
    category_counts[cat] += 1

In [65]:
category_counts_list = [(k, v) for k, v in category_counts.items()]
for (k,v) in sorted(category_counts_list, key = lambda x: -1*x[1]):
    print("{:<25} : {}".format(k, v))

Restaurants               : 51613
Shopping                  : 24595
Food                      : 23014
Beauty & Spas             : 15139
Home Services             : 13202
Health & Medical          : 12033
Nightlife                 : 11364
Bars                      : 9868
Automotive                : 9476
Local Services            : 9343
Event Planning & Services : 8038
Active Life               : 7427
Fashion                   : 6299
Sandwiches                : 5864
Fast Food                 : 5792
American (Traditional)    : 5737
Pizza                     : 5652
Coffee & Tea              : 5565
Hair Salons               : 5395
Hotels & Travel           : 5188
Arts & Entertainment      : 5054
Home & Garden             : 4584
Auto Repair               : 4480
Italian                   : 4411
Burgers                   : 4236
Doctors                   : 4124
Breakfast & Brunch        : 4103
Mexican                   : 3913
Nail Salons               : 3884
Professional Services     : 3865
Ame

In [68]:
categ = set(['Restaurants', 'Food', 'Bars', 'Nightlife'])

In [69]:
category_sets = defaultdict(set)
for index, row in df_cat.iterrows():
    if row['category'] in categ:
        category_sets[row['category']].add(row['business_id'])

In [84]:
len(category_sets['Food'] - category_sets['Restaurants'])

13415

In [85]:
len(category_sets['Nightlife'] - category_sets['Bars'])

1496

In [87]:
len(category_sets['Bars'] - category_sets['Restaurants'])

3178